### 3-3-Visualisation.ipynb 

In [1]:
import os
import cv2
import numpy as np
import seaborn as sns
import rasterio.features
from copy import deepcopy
import matplotlib.pyplot as plt
from shapely import Polygon
from pathlib import Path
from tqdm import tqdm

In [2]:
# Define your base working directory
base_dir = Path("/mnt/c/Projects/Master/Data/Processed/LiveBees/8-SemanticSegmentation")

# Build directory paths from the base directory
pred_fps = list((base_dir / "predictions_resized").glob("*"))
wing_fp = base_dir / "to_predict" / "images"
out_fp = base_dir / "predictions_resized_overlaid"

In [3]:
# Plot overlaid predictions

DEBUG = False

# Load color palette
colors = (np.array(sns.color_palette("hls", 8)) * 255)
colors = colors[[5, 0, 1, 2, 3, 4]]

# Loop through every prediction
for pred_fp in tqdm(pred_fps, desc="Processing files", ncols=145):
    # Determine file paths
    pred_fp = Path(pred_fp)
    pred_fn = pred_fp.name
    seg_fp = Path(wing_fp) / pred_fn
    new_fp = Path(out_fp) / pred_fn

    # Skip if output file exists
    if os.path.exists(new_fp):
        continue  
        
    # Through an error if no matching image exists
    if not os.path.exists(seg_fp):
        print("Warning: Missing Image!")
        continue

    # Open the matching image pair
    seg = cv2.imread(seg_fp)
    seg = cv2.cvtColor(seg, cv2.COLOR_BGR2RGB)
    pred = cv2.imread(pred_fp, cv2.IMREAD_GRAYSCALE)
    overlaid = deepcopy(seg)
    
    try:
        for class_id, color in zip(range(1,7), colors):
            color = tuple(color.astype("float"))
            if (pred == class_id).sum() > 0:
                contours, hierarchy = cv2.findContours((pred == class_id).astype("uint8"), 1, 2)
                area_sorted_indices = np.argsort([cv2.contourArea(x) for x in contours])
                biggest_contour_index = area_sorted_indices[-1]
                biggest_contour = contours[biggest_contour_index].squeeze()
                overlaid = cv2.drawContours(overlaid, [biggest_contour], 0, color, -1)

        # Save the overlaid prediction
        os.makedirs(out_fp, exist_ok=True)
        plt.imshow(seg)  
        plt.imshow(overlaid, alpha=0.5)  
        plt.axis("off")
        plt.savefig(new_fp, bbox_inches="tight", pad_inches=0)
        if DEBUG:
            plt.show()
        plt.close()
        
    except cv2.error as e:
        print(f'Warning: Skipping {pred_fn}\n{e}')

Processing files:  92%|██████████████████████████████████████████████████████████████████████████████▎      | 5134/5574 [00:10<00:01, 384.48it/s]

OpenCV(4.10.0) /home/conda/feedstock_root/build_artifacts/libopencv_1723431247556/work/modules/imgproc/src/drawing.cpp:2433: error: (-215:Assertion failed) p.checkVector(2, CV_32S) >= 0 in function 'fillPoly'



Processing files: 100%|█████████████████████████████████████████████████████████████████████████████████████| 5574/5574 [00:23<00:00, 232.58it/s]
